In [10]:
import pandas as pd
from transformers import pipeline
from datetime import datetime

In [11]:
data = pd.read_csv('../datasets/medical_records.csv')

# Initialize NER and summarization pipelines
ner = pipeline('ner')
summarizer = pipeline('summarization')

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.
No model was supplied, defaulted to google-t5/t5-small and revision d769bba (https://huggingface.co/google-t5/t5-small).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the Py

In [12]:
# Function to preprocess dates and names
def preprocess_data(df):
    # Convert 'Date' to datetime
    df['Visit Date'] = pd.to_datetime(df['Visit Date'], errors='coerce')
    # Drop rows with missing 'Date' or 'Notes'
    df.dropna(subset=['Visit Date', 'Medical Notes'], inplace=True)
    return df

In [13]:
# Function to find records by name using NER
def find_records_by_name(df, name):
    relevant_records = []
    for index, row in df.iterrows():
        entities = ner(row['Medical Notes'])
        for entity in entities:
            if entity['entity'] == 'B-PER' and name.lower() in entity['word'].lower():
                relevant_records.append(row)
                break
    return pd.DataFrame(relevant_records)

In [14]:
# Function to summarize records
def summarize_records(records):
    summaries = []
    for _, record in records.iterrows():
        summary = summarizer(record['Medical Notes'], max_length=100, min_length=50, do_sample=False)[0]['summary_text']
        formatted_date = record['Visited Date'].strftime('%Y-%m-%d')
        summaries.append(f"{formatted_date}: {summary}")
    return summaries

In [15]:
# Example usage
name_to_search = "Ian Lee"
data = preprocess_data(data)
relevant_records = find_records_by_name(data, name_to_search)
summaries = summarize_records(relevant_records)

for summary in summaries:
    print(summary)

KeyboardInterrupt: 